In [1]:
# ESta version solo procesa el dataset y da mensajes 
# No visualiza ninguna imagen

In [2]:
# --- Importar las librerías necesarias ---

import torch
from torch.utils.data import Dataset, DataLoader
import os
import cv2
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pandas as pd
from sklearn.model_selection import train_test_split


In [3]:
# PASO 1: Definición de la clase BloodCellDataset
# Esta clase debe manejar la carga de imágenes y anotaciones, así como las transformaciones necesarias.

class BloodCellDataset(Dataset):
    def __init__(self, data_root, annotations_df, image_size=(416, 416), transform=None):
        """
        Args:
            data_root (str): Ruta al directorio raíz que contiene la carpeta 'BCCD' (tus imágenes).
            annotations_df (pd.DataFrame): DataFrame de pandas con las anotaciones ya cargadas y pre-filtradas para este split (train/val/test).
            image_size (tuple): Tamaño (ancho, alto) al que se redimensionarán las imágenes para el modelo.
            transform (albumentations.Compose, optional): Transformaciones a aplicar.
        """
        self.data_root = data_root
        self.image_folder = os.path.join(data_root, 'BCCD') # Asumiendo que las imágenes están en data_root/BCCD
        self.image_size = image_size
        self.transform = transform
        
        # Mapeo de nombres de clase a IDs numéricos (¡AJUSTA EL ORDEN SI ES NECESARIO!)
        self.class_name_to_id = {
            'RBC': 0,        # Glóbulo Rojo
            'WBC': 1,        # Glóbulo Blanco
            'Platelets': 2   # Plaquetas
        }
        
        # --- NUEVO: Pre-procesar el DataFrame para este split ---
        # Agrupar anotaciones por filename
        self.image_annotations = {}
        for filename, group in annotations_df.groupby('filename'):
            bboxes_pixel_list = []
            for idx, row in group.iterrows():
                cell_type = row['cell_type']
                xmin = int(row['xmin'])
                xmax = int(row['xmax'])
                ymin = int(row['ymin'])
                ymax = int(row['ymax'])
                
                class_id = self.class_name_to_id.get(cell_type)
                if class_id is None:
                    print(f"Advertencia: Tipo de célula desconocido '{cell_type}' en el archivo {filename}. Saltando anotación.")
                    continue

                bboxes_pixel_list.append([xmin, ymin, xmax, ymax, class_id])
            self.image_annotations[filename] = bboxes_pixel_list
        
        self.image_files = list(self.image_annotations.keys()) # Lista de imágenes para este split
        print(f"Dataset inicializado con {len(self.image_files)} imágenes.")
        
    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_folder, img_name)
        
        image = cv2.imread(img_path)
        if image is None:
            raise FileNotFoundError(f"No se pudo cargar la imagen: {img_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        original_h, original_w, _ = image.shape

        bboxes_pixel = self.image_annotations.get(img_name, [])
        
        bboxes = []
        class_labels = []
        for bbox_px in bboxes_pixel:
            xmin_px, ymin_px, xmax_px, ymax_px, class_id = bbox_px
            
            # Normalizar las coordenadas
            xmin_norm = xmin_px / original_w
            ymin_norm = ymin_px / original_h
            xmax_norm = xmax_px / original_w
            ymax_norm = ymax_px / original_h
            
            bboxes.append([xmin_norm, ymin_norm, xmax_norm, ymax_norm])
            class_labels.append(class_id)
        
        if self.transform:
            transformed = self.transform(image=image, bboxes=bboxes, class_labels=class_labels)
            image = transformed['image']
            bboxes = transformed['bboxes']
            class_labels = transformed['class_labels']
            
        if not isinstance(image, torch.Tensor):
            image = torch.from_numpy(image).permute(2, 0, 1).float() / 255.0

        yolo_bboxes = []
        for i, bbox in enumerate(bboxes):
            x_min, y_min, x_max, y_max = bbox
            
            x_min = max(0.0, min(1.0, x_min))
            y_min = max(0.0, min(1.0, y_min))
            x_max = max(0.0, min(1.0, x_max))
            y_max = max(0.0, min(1.0, y_max))

            center_x = (x_min + x_max) / 2
            center_y = (y_min + y_max) / 2
            width = x_max - x_min
            height = y_max - y_min
            
            if width <= 0 or height <= 0:
                continue 

            yolo_bboxes.append([class_labels[i], center_x, center_y, width, height])
            
        if len(yolo_bboxes) == 0:
            yolo_bboxes = torch.zeros((0, 5), dtype=torch.float32)
        else:
            yolo_bboxes = torch.tensor(yolo_bboxes, dtype=torch.float32)
        
        return image, yolo_bboxes


In [4]:
# PASO 2: Definición de las transformaciones de Albumentations
# Define el tamaño de entrada de tu modelo YOLOv3 (416x416)

# Define el tamaño de entrada de tu modelo YOLOv3 (416x416)
YOLO_INPUT_SIZE = (416, 416) 

train_transforms = A.Compose([
    A.LongestMaxSize(max_size=YOLO_INPUT_SIZE[0], p=1.0), 
    A.PadIfNeeded(min_height=YOLO_INPUT_SIZE[0], min_width=YOLO_INPUT_SIZE[1], border_mode=cv2.BORDER_CONSTANT, value=0, p=1.0),
    A.RandomCrop(height=YOLO_INPUT_SIZE[1], width=YOLO_INPUT_SIZE[0], p=0.8),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2), 
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5, border_mode=cv2.BORDER_CONSTANT, value=0   ), 
    A.RGBShift(r_shift_limit=10, g_shift_limit=10, b_shift_limit=10, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.GaussNoise(p=0.2),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(), 
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels'])) 


val_test_transforms = A.Compose([
    A.LongestMaxSize(max_size=YOLO_INPUT_SIZE[0], p=1.0), 
    A.PadIfNeeded(min_height=YOLO_INPUT_SIZE[0], min_width=YOLO_INPUT_SIZE[1], border_mode=cv2.BORDER_CONSTANT, value=0, p=1.0),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))


In [5]:
# PASO 3: Definición de la función Collate_fn para el DataLoader
def collate_fn(batch):
    images = []
    bboxes = []
    for img, bbox_target in batch:
        images.append(img)
        bboxes.append(bbox_target) 
    images = torch.stack(images, 0)
    return images, bboxes

In [6]:
# Directorio en los que estan almacenadas las imagenes y las anotaciones 
# C:\Users\gtoma\Master_AI_Aplicada\GitHubRep\PyTorch-YOLOv3\dataset

In [7]:
# PASO 4: Definición de la Lógica de División del Dataset y Creación de DataLoaders

if __name__ == '__main__':
    # RUTAS A LOS DATOS
    DATA_ROOT = 'C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/dataset'
    CSV_FILE = 'C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/dataset/annotations.csv'
    
    # Parámetros de la división en train/val/test
    TEST_SPLIT_RATIO = 0.15   # 15% para el conjunto de prueba
    VAL_SPLIT_RATIO = 0.15    # 15% para el conjunto de validación (del resto de los datos)
    
    # Semilla para la división aleatoria
    RANDOM_SEED = 1234        # Semilla para la reproducibilidad de la división

    # Parámetros del DataLoader
    BATCH_SIZE = 8
    NUM_WORKERS = 0 # Deja en 0 para depuración, luego puedes aumentarlo a 4-8

    # Cargando todas las anotaciones y obteniendo nombres de archivo únicos
    print(f"Cargando todas las anotaciones desde: {CSV_FILE}")
    full_df = pd.read_csv(CSV_FILE)
    
    # Obteniendo la lista de nombres de archivo únicos presentes en el CSV
    all_image_filenames = full_df['filename'].unique().tolist()
    print(f"Total de {len(all_image_filenames)} imágenes únicas encontradas en el CSV.")

    # Dividiendo los nombres de archivo en entrenamiento y test
    # Primero separamos el conjunto de prueba
    train_val_filenames, test_filenames = train_test_split(
        all_image_filenames, 
        test_size=TEST_SPLIT_RATIO, 
        random_state=RANDOM_SEED
    )
    
    # Luego, dividimos el conjunto de entrenamiento/validación en entrenamiento y validación
    train_filenames, val_filenames = train_test_split(
        train_val_filenames, 
        test_size=VAL_SPLIT_RATIO / (1 - TEST_SPLIT_RATIO), # Proporción relativa al conjunto restante
        random_state=RANDOM_SEED
    )

    print(f"Imágenes para entrenamiento: {len(train_filenames)}")
    print(f"Imágenes para validación: {len(val_filenames)}")
    print(f"Imágenes para prueba: {len(test_filenames)}")

    # Creamos DataFrames de anotaciones para cada split
    train_df = full_df[full_df['filename'].isin(train_filenames)].copy()
    val_df = full_df[full_df['filename'].isin(val_filenames)].copy()
    test_df = full_df[full_df['filename'].isin(test_filenames)].copy()

    # Creamos instancias del Dataset y DataLoader para cada split
    train_dataset = BloodCellDataset(
        data_root=DATA_ROOT,
        annotations_df=train_df, # Pasar el DataFrame filtrado
        image_size=YOLO_INPUT_SIZE,
        transform=train_transforms
    )
    val_dataset = BloodCellDataset(
        data_root=DATA_ROOT,
        annotations_df=val_df, # Pasar el DataFrame filtrado
        image_size=YOLO_INPUT_SIZE,
        transform=val_test_transforms # Usar transformaciones sin aumento para validación
    )
    test_dataset = BloodCellDataset(
        data_root=DATA_ROOT,
        annotations_df=test_df, # Pasar el DataFrame filtrado
        image_size=YOLO_INPUT_SIZE,
        transform=val_test_transforms # Usar transformaciones sin aumento para prueba
    )

    train_dataloader = DataLoader(
        train_dataset, batch_size=BATCH_SIZE, shuffle=True,
        num_workers=NUM_WORKERS, collate_fn=collate_fn, pin_memory=True
    )
    val_dataloader = DataLoader(
        val_dataset, batch_size=BATCH_SIZE, shuffle=False,
        num_workers=NUM_WORKERS, collate_fn=collate_fn, pin_memory=True
    )
    test_dataloader = DataLoader(
        test_dataset, batch_size=BATCH_SIZE, shuffle=False,
        num_workers=NUM_WORKERS, collate_fn=collate_fn, pin_memory=True
    )

    # Verificamos la carga de un lote de entrenamiento
    print("\nVerificando la carga de un lote de entrenamiento...")
    for images, targets in train_dataloader:
        print(f"Tamaño del lote de imágenes: {images.shape}")
        print(f"Número de imágenes en el lote (lista de targets): {len(targets)}") 
        if len(targets[0]) > 0:
            print(f"Ejemplo de target para la primera imagen (clase, cx, cy, w, h normalizados):")
            print(targets[0][0]) 
        else:
            print("Primera imagen del lote de entrenamiento sin cajas anotadas (o filtradas).")
        break 

    print("\nDataset y DataLoaders de entrenamiento, validación y prueba configurados exitosamente.")

Cargando todas las anotaciones desde: C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/dataset/annotations.csv
Total de 364 imágenes únicas encontradas en el CSV.
Imágenes para entrenamiento: 254
Imágenes para validación: 55
Imágenes para prueba: 55
Dataset inicializado con 254 imágenes.
Dataset inicializado con 55 imágenes.
Dataset inicializado con 55 imágenes.

Verificando la carga de un lote de entrenamiento...
Tamaño del lote de imágenes: torch.Size([8, 3, 416, 416])
Número de imágenes en el lote (lista de targets): 8
Primera imagen del lote de entrenamiento sin cajas anotadas (o filtradas).

Dataset y DataLoaders de entrenamiento, validación y prueba configurados exitosamente.
